<a id='Top'></a>

# Data Wrangling


                                                   By: Jun Ho Lee
___


<a id='Table of Contents'></a>
## Table of Contents

1. <a href='#Initial Importing Steps'>Initial Importing Steps</a>
2. <a href='#Initial Inspection'>Initial Inspection</a>
3. <a href='#Dataset Cleaning 1'>Dataset Cleaning 1 - Cleaning Columns</a>
    1. <a href='#Dataset Cleaning 1'>Drop columns with missing values</a>
    2. <a href='#Dataset Cleaning 1b.1'>Drop unnecessary columns - Agent/Attorney</a>
    3. <a href='#Dataset Cleaning 1b.2'>Drop unnecessary columns - Prevailing Wage</a>
    4. <a href='#Dataset Cleaning 1c'>Remove redundant columns - Area</a>
4. <a href='#Numeric Columns'>Columns that should be numeric</a>
5. <a href='#Dataset Cleaning 2'>Dataset Cleaning 2 - Cleaning Rows (Imputation)</a>
    1. <a href='#Dataset Cleaning 2'>Imputing values for WAGE column</a>
        1. <a href='#Prevailing Wage'>Note on PREVAILING-WAGE</a>
    2. <a href='#Dataset Cleaning 2b'>Investigating Missing Values (H1B-DEPENDENT & WILLFUL-VIOLATOR columns)</a>
    3. <a href='#Dataset Cleaning 2c'>Imputing values based on existing column relationship - STATE & POSTAL-CODE</a>
    4. <a href='#Dataset Cleaning 2d'>POSTAL-CODES that are *"weird"*</a>
    5. <a href='#Dataset Cleaning 2e'>Final Missing Value Handling</a>

<a href='#Top'>Back to Top</a>

<a id='Initial Importing Steps'></a>
____
### 1. Initial Importing Steps

Using the column information saved in the JSON file, we will load in the dataset so that memory usage efficiency can be improved. The initial loading step would be done for every subsequent new notebooks but the steps will be only detailed in this current ***DATA WRANGLING*** notebook. In future notebooks, the initial dataset importing step will be wrapped in a function.


**A. Import Necessary Libraries:**

In [1]:
import numpy as np
import pandas as pd
# # set option to see all the columns
pd.set_option('display.max_columns', 50)

import matplotlib.pyplot as plt
import seaborn as sns

**B. Previous function to check memory usage of new dataframe** 

Adapted from [Dataquest](https://www.dataquest.io/blog/pandas-big-data/)

In [2]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

**C. Import JSON file and extract column information**

In [3]:
# Import JSON file for column information extraction 
import json

with open('data/h1b_df_params.json', 'r') as fp:
    h1b_params = json.load(fp)

final_cols = h1b_params['c_final']
column_types = h1b_params['c_type']
date_columns = h1b_params['c_date']

**D. Read in the original dataframe with optimized column parameters** 

In [4]:
# Don't RUN this TWICE!! 
h1b_df = pd.read_csv("data/H-1B_Disclosure_Data_FY2018_EOY.csv", usecols=final_cols, dtype=column_types, low_memory=False, parse_dates=date_columns,infer_datetime_format=True)

**E. Check memory usage to verify optimization**

In [5]:
# Check Memory Usage 
print(mem_usage(h1b_df))
print("Initial dimensinos of the dataframe: {}".format(h1b_df.shape))

204.63 MB
Initial dimensinos of the dataframe: (654360, 50)


<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Initial Inspection'></a>
___
### 2. Initial Inspection

Now that the dataset is read in, let's take a look at the shape of the dataset. Initial inspection (`h1b_df.shape`) tells us that the current dataset has a total of 50 columns and 654360 rows. Let's dive in deeper with `.info()` method.

In [6]:
# # Commented out due to space constraints
# h1b_df.info(memory_usage='deep')

___
**From the initial inspection, three things stand out:**
1. There are a number of columns that have a lot of missing values
2. There are some columns that are probably not useful in predicting the outcome variable (`CASE_STATUS`). I would need to make a judgment call on this matter.
3. There are a few columns whose information are redundant and can probably be explained by another existing variable (ex. `EMPLOYER_POSTAL_CODE` is a unique code that can explain `CITY`, `ADDRESS` information etc.) 


To clean our dataset and reduce colinearity among the features for future machine learning pipeline, we will identify these columns and drop them from the original dataframe. To identify columns with **'many'** missing values, I will set a threshold of 50% and drop those columns that are above the threshold. 

After cleaning the columns let's take another look at the dataset.  


<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 1'></a>
___
### 3. Dataset Cleaning 1 - Cleaning Columns

### 3a. Drop columns with missing values
- As a first step, I will be dropping columns that have more than 50% of their rows missing since those rows will not provide any useful information on predicting the outcome variable (`CASE_STATUS`)

- I will be saving the new dataframe to `h1b_df_clean` and updating (dropping) the dataframe columns sequentially throughout this notebook.

**A. Function to determine missing value percentage**

- Modified from [Dametreusv Github](https://github.com/dametreusv/world_development_indicators_data_science/blob/master/WDI_wrangle.ipynb)

In [7]:
# # Modified from https://github.com/dametreusv/world_development_indicators_data_science/blob/master/WDI_wrangle.ipynb

def find_cols_to_drop(df):
    cols_to_drop = []
    columns = df.columns
    for column in columns:
        percentage = (df[column].isnull().sum() / df.shape[0]) * 100
    
        # append columns to "drop_list" if the missing value percentage is greater than 50% 
        if percentage > 50:
            cols_to_drop.append(column)
        
    return cols_to_drop
        

**B. Drop the columns that meet the above criteria**

In [8]:
drop_cols = find_cols_to_drop(h1b_df)
h1b_df_clean = h1b_df.drop(drop_cols, axis=1)

**C. Cleaned Dataframe Dimensions**

In [9]:
h1b_df_clean.shape

(654360, 44)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 1b.1'></a>
___
### 3b. Drop unnecessary columns - Part 1 (Agent/Attorney)

- For the second step, I will be making a judgment call based on domain knowledge. My end goal is to predict `CASE_STATUS` given our predictor variables. Given this end goal, I believe information on `AGENT_ATTORNEY` will not be useful at all with the exception of `AGENT_REPRESENTING_EMPLOYER`, which contains information on whether the employer was represented / not represented by an agent or an attorney. Therefore we will drop the columns that have information on `AGENT_ATTORNEY`.

**A. Selecting columns that contain 'Agent/Attorney'**

In [10]:
# selecting column names that contain 'AGENT ATTORNEY'
cols = h1b_df_clean.columns
attorney_bool = cols.str.contains("AGENT_ATTORNEY")
attorney_idx = np.where(attorney_bool)[0]

**B. Drop the columns that meet the above criteria**

In [11]:
drop_agent_cols = cols[attorney_idx]
h1b_df_clean2 = h1b_df_clean.drop(drop_agent_cols, axis=1)

**C. Cleaned Dataframe Dimensions**

In [12]:
h1b_df_clean2.shape

(654360, 41)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 1b.2'></a>
___
### 3b. Drop unnecessary columns - Part 2 (Prevailing Wage)

- The h1b dataframe also has a column called `PREVAILING WAGE` and other columns based on `PREVAILING WAGE`. Looking at the data dictionary however, it is difficult to understand what `PW_SOURCE` signifies. The variables for this column ("OES" / "CBA" / "DBA" / "SCA" / "Other") are not explained at all. Therefore I will drop all columns associated with `PW_SOURCE` since the variables would not be interpretable. 

- I will additionally drop the `PW_WAGE_LEVEL` column since I can create custom wage levels based on the `PREVAILING_WAGE` column. `PW_WAGE_LEVEL` information would be redundant. 


**A. Selecting columns that contain 'PW_Source' and 'PW_Wage_Level**

In [13]:
# selecting column names that contain 'PW_SOURCE' and 'PW_WAGE_LEVEL' 
cols = h1b_df_clean2.columns
pw_bool = cols.str.contains("PW_SOURCE|PW_WAGE_LEVEL")
pw_idx = np.where(pw_bool)[0]

**B. Drop the columns that meet the above criteria**

In [14]:
drop_pw_cols = cols[pw_idx]
h1b_df_clean3 = h1b_df_clean2.drop(drop_pw_cols, axis=1)

**C. Cleaned Dataframe Dimensions**

In [15]:
h1b_df_clean3.shape

(654360, 37)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 1c'></a>
___
### 3c. Remove redundant columns

- For the last step, I will also be making a judgment call. I know that the ***POSTAL_CODE*** can uniquely describe every area of a country and thus will describe *ADDRESS, CITY, PROVINCE, COUNTRY, STATE* information as well. Therefore we can solely retain the ***POSTAL_CODE*** column and still capture the entire area information. This comes at a cost however, since ***POSTAL_CODE*** information is very granular. Therefore I will still retain *STATE* information in the `cleaned dataset` to observe any trends in the analysis. Moreover, I will save all the area columns (including ***POSTAL_CODE*** column) into a separate dataframe so that I can query into the arae dataframe and come back later for investigation if necessary. The cleaned dataframe will have ***POSTAL_CODE*** and ***STATE*** columns.

**A. Selecting columns that contain AREA information**

In [16]:
# # selecting column names that contain 'ADDRESS|COUNTY|CITY|PROVINCE|STATE|POSTAL_CODE|COUNTRY'
# # Note that 'PROVINCE' column was deleted from 3A

cols = h1b_df_clean3.columns
area_bool = cols.str.contains("ADDRESS|COUNTY|CITY|PROVINCE|STATE|POSTAL_CODE|COUNTRY")
area_idx = np.where(area_bool)[0]
area_cols = cols[area_idx]

**B. Create new `area dataframe` and save it as a csv file along with its dtypes**

*Previous function that returns column dtypes* 
- Needed later when reading in `area_df` from a different notebook

In [17]:
def save_col_dtypes(df):
    dtypes = df.dtypes

    col_name = dtypes.index                      # name of the column 
    col_types = [i.name for i in dtypes.values]  # datatype of the column

    # # Save column name and types into a dictionary 
    # # {name: type}
    column_types = dict(zip(col_name, col_types))
    return column_types

*Saving the `area_df` to csv file*

In [18]:
area_cols = cols[area_idx]
area_df = h1b_df_clean3[area_cols]
area_df.to_csv("data/area_df", index=False)

*Save column types into a json format*

In [19]:
area_col_types = save_col_dtypes(area_df)

# convert to json format
with open('data/area_df_params.json', 'w') as fp:
    json.dump(area_col_types, fp)  # convert dict as a json file format! 

**C. Drop the area columns except `STATE` and `POSTAL CODE` columns**

In [20]:
# # selecting column names that contain 'ADDRESS|COUNTY|CITY|PROVINCE|COUNTRY'
# # Note that 'PROVINCE' column was deleted from 3A

cols = h1b_df_clean3.columns
drop_area_bool = cols.str.contains("ADDRESS|COUNTY|CITY|PROVINCE|COUNTRY")
drop_area_idx = np.where(drop_area_bool)[0]
drop_area_cols = cols[drop_area_idx]

**D. Drop the columns that meet the above criteria**

In [21]:
h1b_df_clean4 = h1b_df_clean3.drop(drop_area_cols, axis=1)

**E. Cleaned Dataframe Dimensions**

In [22]:
h1b_df_clean4.shape

(654360, 32)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Numeric Columns'></a>
____
### 4. Columns that should be numeric

Now that we cleaned up the columns of the dataframe, let's take another look at the data - this time thinking about whether the **dtypes** we picked for each column were actually valid.

In fact, if we investigate the *dtypes* for columns `PREVAILING_WAGE`, `WAGE_RATE_OF_PAY_FROM`, `WAGE_RATE_OF_PAY_TO`, they are either *Categorical* or *Object* dtypes. This doesn't make sense since wage is always expressed in numbers and thus would have to be a numeric *dtype*(`int` or `float`). This suggests that some values in these columns have non-numeric characters such as commas(,) or periods(.). We will clean up these non-numeric characters and convert them to a numeric *dtype*.

**A. Remove non-numeric characters and convert to numeric**

In [23]:
# # Remove commas (non_numeric char) and convert to numeric (float) dtype
# # For three columns
h1b_df_clean4.PREVAILING_WAGE = pd.to_numeric(h1b_df_clean4.PREVAILING_WAGE.str.replace(',',''), downcast='float')
h1b_df_clean4.WAGE_RATE_OF_PAY_FROM = pd.to_numeric(h1b_df_clean4.WAGE_RATE_OF_PAY_FROM.str.replace(',',''),downcast='float')
h1b_df_clean4.WAGE_RATE_OF_PAY_TO = pd.to_numeric(h1b_df_clean4.WAGE_RATE_OF_PAY_TO.str.replace(',',''),downcast='float')

**B. Check datatypes**

In [24]:
# # Commented out due to space constraints
# h1b_df_clean4.info()

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 2'></a>
____
### 5. Dataset Cleaning 2 - Cleaning Rows (Imputation)

### 5a. Imputing `WAGE_RATE_OF_PAY_TO` Column

Now that we cleaned up the columns of the dataframe and converted necessary columns to more suitable datatypes, let's take another look at the data - this time focusing on the rows. If there are any missing or unordinary values, we would need to impute and replace them with more suitable values. 

Let's actually first take a look at the first five rows of the three columns that we converted to numeric dtype.

**A. Initial Investigation**

In [25]:
numeric_df = h1b_df_clean4.loc[:,['PREVAILING_WAGE', 'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO']]
numeric_df.head()
# numeric_df.isnull().sum()

,PREVAILING_WAGE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO
0,112549.0,143915.0,0.0
1,79976.0,100000.0,0.0
2,77792.0,78240.0,0.0
3,84406.0,84406.0,85000.0
4,87714.0,95000.0,0.0


**Result:** Upon inspection, there are a lot of 0s in the  `WAGE_RATE_OF_PAY_TO` column. Let's actually see how many zeros there are in this column. 

In [26]:
# Returns the length (number) of zeros in this particular column
n_of_zeros_in_column = len(np.where(numeric_df.WAGE_RATE_OF_PAY_TO == 0)[0])
print("# of zeros in WAGE_RATE_OF_PAY_TO column: {}".format(n_of_zeros_in_column))

n_of_nan_in_column = numeric_df.WAGE_RATE_OF_PAY_TO.isnull().sum()
print("# of nan in WAGE_RATE_OF_PAY_TO column: {}".format(n_of_nan_in_column))

to_null_idx = np.where(numeric_df.WAGE_RATE_OF_PAY_TO.isnull())
numeric_df.loc[to_null_idx] # print out the dataframe  of only nan values 

# of zeros in WAGE_RATE_OF_PAY_TO column: 511240
# of nan in WAGE_RATE_OF_PAY_TO column: 3


,PREVAILING_WAGE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO
119,26.280001,26.280001,NaN
31918,26.280001,26.280001,NaN
34803,72842.000000,75000.000000,NaN


**Result:** It looks like majority of the values are zeros and only three values are `nan` values. Further inspection reveals that the rows at which `WAGE_RATE_OF_PAY_TO` (TO) are nan values do have values for `PREVAILING WAGE` and `WAGE_RATE_OF_PAY_FROM` (FROM). It looks like zeros were inputted as substitutes for `nan` values. Since the zeros can be essentially be thought as `nan` values, we would need to impute accordingly. 

The non-zero values of `TO` are typically greater than `FROM` values. Thus it would be logical to impute `TO` values based on `FROM` values. If `TO` values are zero or `nan` values, I will replace them with the values from the `FROM` column.

Additionally, if any values from the `FROM` column are zeros, then that means those values are essentially `nan` values. I will drop those rows as a last step. 



**B. Imputing `WAGE_RATE_OF_PAY_TO` column**

In [27]:
# # # Remember to perform operation on the entire cleaned h1b dataset
# # # NOT the numeric_df, which was just used for testing purposes. 

# # Remember that .loc returns a group of rows!
# # If any values from `TO` column are zeros, replace with the value from `FROM` column
h1b_df_clean4.loc[h1b_df_clean4['WAGE_RATE_OF_PAY_TO'] == 0, ['WAGE_RATE_OF_PAY_TO']] = h1b_df_clean4['WAGE_RATE_OF_PAY_FROM']


# # If any values from `TO` column are nan, replace with the value from `FROM` column
# # Can't use | operator with nans
h1b_df_clean4.loc[np.isnan(h1b_df_clean4.WAGE_RATE_OF_PAY_TO), ['WAGE_RATE_OF_PAY_TO']] = h1b_df_clean4['WAGE_RATE_OF_PAY_FROM']


**C. Drop rows where values of `FROM` are zeros**

In [28]:
# # First check the number of zeros in `FROM` column
n_of_zeros_in_from = len(np.where(h1b_df_clean4.WAGE_RATE_OF_PAY_FROM == 0)[0])
print("# of zeros in WAGE_RATE_OF_PAY_FROM column: {}".format(n_of_zeros_in_from))


# of zeros in WAGE_RATE_OF_PAY_FROM column: 12


In [29]:
# # Get the index where the values are zero
zero_idx = np.where(h1b_df_clean4.WAGE_RATE_OF_PAY_FROM == 0)[0]

# # observe the dataframe
h1b_df_clean4.loc[zero_idx]

# # Drop the row indices (DON'T USE inplace operation!!) and set a new dataframe
h1b_df_clean5 = h1b_df_clean4.drop(zero_idx)

# # RESET the index so that the dropped indices don't affect future calculations
# # VERY VERY IMPT!
h1b_df_clean6 = h1b_df_clean5.reset_index(drop=True)


**NOTE:** Resetting the index after dropping the rows is CRUCIAL. If you don't reset the index, the dataframe will retain the original index but internally assign a new index in future manipulations. Thus subsequent indexing will be errorneous!! 

Ex.)  
DataFrame with index column = [0, 1, 2, 3, 4, 5] <br>
Original Index --> 0, 1, 2, 3, 4, 5 <br>
Dropped Index --> 3 <br>
Retained Index --> 0, 1, 2,    4, 5 <br>
Future Manipulation (index 3 --> returns 4!!) (since internally re-indexes from the start)


     


In [30]:
# # Note the Differences between the two dataframes below!!!!! 
# # TestTestTest
h1b_df_clean5.iloc[688:691]  # before resetting index
h1b_df_clean6.iloc[688:691]  # after resetting index

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,AGENT_REPRESENTING_EMPLOYER,JOB_TITLE,SOC_CODE,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMP,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,WORKSITE_POSTAL_CODE
688,I-200-18243-981567,CERTIFIED,2018-08-31,2018-09-07,H-1B,2018-09-24,2021-09-23,"ADVANCED MICRO DEVICES, INC.",CA,95054,Y,HR BUSINESS PARTNER MANAGER,11-3121,HUMAN RESOURCES MANAGERS,334413.0,1,0,0,0,0,1,0,Y,131972.000000,Year,131972.000000,178282.000000,Year,N,N,TX,78735
689,I-200-18243-060543,CERTIFIED,2018-08-31,2018-09-07,H-1B,2018-08-31,2021-08-31,"MASTECH DIGITAL TECHNOLOGIES, INC., A MASTECH ...",PA,15108,N,SENIOR SOFTWARE DEVELOPER,15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",541511.0,6,1,1,1,1,1,1,Y,50.619999,Hour,50.619999,50.619999,Hour,Y,N,PA,19103
690,I-200-18239-164083,CERTIFIED,2018-08-27,2018-08-31,H-1B,2018-09-20,2020-09-19,UNIVERSITY OF WASHINGTON,WA,98195,N,RESEARCH ASSOCIATE,19-1021,BIOCHEMISTS AND BIOPHYSICISTS,611310.0,1,1,0,0,0,0,0,Y,41579.000000,Year,48432.000000,48432.000000,Year,N,N,WA,98195


**D. Verifying the Imputation**

In [31]:
# # Verifying that `FROM` column doesn't contain any zero values
count_zero = len(np.where(h1b_df_clean5.WAGE_RATE_OF_PAY_FROM == 0)[0])
print("Number of cells with a value of zero in `FROM` column: {}".format(count_zero))

# # Verifying the imputation worked as expected
h1b_df_clean5[['PREVAILING_WAGE','WAGE_RATE_OF_PAY_TO','WAGE_RATE_OF_PAY_FROM']].head(3)

Number of cells with a value of zero in `FROM` column: 0


,PREVAILING_WAGE,WAGE_RATE_OF_PAY_TO,WAGE_RATE_OF_PAY_FROM
0,112549.0,143915.0,143915.0
1,79976.0,100000.0,100000.0
2,77792.0,78240.0,78240.0


**E. New Dataframe Dimensions**

In [32]:
# # Note the change in rows
h1b_df_clean6.shape

(654348, 32)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Prevailing Wage'></a>
____

#### Note on `PREVAILING_WAGE`

By the above logic in <a href='#Dataset Cleaning 2'>5a</a>, I could have regarded the zero values in PREVAILING_WAGE as intrinsically missing values and converted to non-zero integers as well. There is however a major reason why I didn't perform this manipulation and decided to drop these rows instead. 

1. It's really hard to accurately impute a value for `PREVAILING WAGE` due to the fact that every labor sub-area has a different prevailing wage. Therefore the typical descriptive statistics method to fill in missing values will not work. 

Therefore, we will be dropping the rows where the values are either zero or `nan` in `PREVAILING WAGE` column.


**- Dropping zero values**

In [33]:
# # Check how many rows have zero values in PREVAILING WAGE column
len(np.where(h1b_df_clean6.PREVAILING_WAGE == 0)[0])

# # Check how many rows have nan values in PREVAILING WAGE column
len(np.where(h1b_df_clean6.PREVAILING_WAGE.isnull())[0])

# # Getting the index where value is 0
wage_zero_idx = np.where(h1b_df_clean6.PREVAILING_WAGE == 0)[0]
# h1b_df_clean5.loc[wage_zero_idx]  # investigating the indices

In [34]:
# # Example of 'zero_idx'
wage_zero_idx

array([  2458,   6193,   6775,   9724,   9739,   9839,  14599,  14731,
        17007,  17011,  17508,  18725,  19368,  20568,  20873,  21035,
        21321,  21530,  21611,  21803,  22184,  25268,  25304,  25633,
        25725,  26968,  28906,  29018,  29487,  30074,  30723,  31312,
        31551,  31607,  32244,  32785,  33506,  33984,  37300,  37653,
        38322,  39653,  42355,  46111,  46563,  47516,  49081,  51927,
        53364,  53866,  54012,  54132,  57147,  57698,  60523,  61749,
       186618, 255833, 601816])

**NOTE**: Be careful with dealing with row-wise missing values! Always remember to reset_index and don't use inplace operations! (It will push back the original index and propagate any indexed-based filteration)

In [35]:
# # Drop the zero-value indices and reset the index! 
h1b_df_clean7 = h1b_df_clean6.drop(wage_zero_idx)
h1b_df_clean8 = h1b_df_clean7.reset_index(drop=True)

h1b_df_clean8.shape

(654289, 32)

**- Dropping null values**

In [36]:
# # Getting the index where it is null (get_index should be done AFTER dropping/resetting_index)
# # With the most recent cleaned dataframe
wage_null_idx = np.where(h1b_df_clean8.PREVAILING_WAGE.isnull())[0]
# h1b_df_clean5.loc[wage_null_idx]  # investigating the indices

# # Drop the null indices and reset the index! 
h1b_df_clean9 = h1b_df_clean8.drop(wage_null_idx)
h1b_df_clean10 = h1b_df_clean9.reset_index(drop=True)

h1b_df_clean10.shape

(654286, 32)

**NOTE**:  
*For more information / investigation, play with the below dataframes*  
Try chaning the suffix # to different values (6~10)

In [37]:
# # Example regardin zero-value index
# h1b_df_clean9.loc[2457:2460, ['PREVAILING_WAGE','PW_UNIT_OF_PAY']]


**- Verification Step**

In [38]:
# # Verifying that `PREVAILING WAGE` column doesn't contain any zero values or null values
pw_zero = len(np.where(h1b_df_clean10.PREVAILING_WAGE == 0)[0])
print("Number of cells with a value of zero in `Prevaling Wage` column: {}".format(pw_zero))

pw_null = len(np.where(h1b_df_clean10.PREVAILING_WAGE.isnull())[0])
print("Number of cells with nan values in `Prevaling Wage` column: {}".format(pw_null))


Number of cells with a value of zero in `Prevaling Wage` column: 0
Number of cells with nan values in `Prevaling Wage` column: 0


<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 2b'></a>
____

### 5b. Investigating Missing Values

Now let's actually take a look at any missing values in the rows. We utilize the `find_cols_to_drop` funtion to investigate any remaining missing values. Using print statements, we can identify which columns still have missing values.

**A. Modified `find_cols_to_drop` Function:**

In [39]:
# # Adapted from https://github.com/dametreusv/world_development_indicators_data_science/blob/master/WDI_wrangle.ipynb

def display_missing_rows(df):
    columns = df.columns
    print('MISSING ROWS per COLUMN')
    for column in columns:
        percentage = (df[column].isnull().sum() / df.shape[0]) * 100
        print('{}: {}, {:0.2f}%'.format(column, df[column].isnull().sum(), percentage))

# # Call the function
# h1b_df_clean4 is the final cleaned df from above 
# (even after value imputation for WAGE column)
display_missing_rows(h1b_df_clean10)

MISSING ROWS per COLUMN
CASE_NUMBER: 0, 0.00%
CASE_STATUS: 0, 0.00%
CASE_SUBMITTED: 1, 0.00%
DECISION_DATE: 0, 0.00%
VISA_CLASS: 0, 0.00%
EMPLOYMENT_START_DATE: 3, 0.00%
EMPLOYMENT_END_DATE: 6, 0.00%
EMPLOYER_NAME: 13, 0.00%
EMPLOYER_STATE: 67, 0.01%
EMPLOYER_POSTAL_CODE: 12, 0.00%
AGENT_REPRESENTING_EMPLOYER: 10, 0.00%
JOB_TITLE: 0, 0.00%
SOC_CODE: 3, 0.00%
SOC_NAME: 3, 0.00%
NAICS_CODE: 1, 0.00%
TOTAL_WORKERS: 0, 0.00%
NEW_EMPLOYMENT: 0, 0.00%
CONTINUED_EMPLOYMENT: 0, 0.00%
CHANGE_PREVIOUS_EMPLOYMENT: 0, 0.00%
NEW_CONCURRENT_EMP: 0, 0.00%
CHANGE_EMPLOYER: 0, 0.00%
AMENDED_PETITION: 0, 0.00%
FULL_TIME_POSITION: 2, 0.00%
PREVAILING_WAGE: 0, 0.00%
PW_UNIT_OF_PAY: 1, 0.00%
WAGE_RATE_OF_PAY_FROM: 0, 0.00%
WAGE_RATE_OF_PAY_TO: 0, 0.00%
WAGE_UNIT_OF_PAY: 3, 0.00%
H1B_DEPENDENT: 14132, 2.16%
WILLFUL_VIOLATOR: 14137, 2.16%
WORKSITE_STATE: 4, 0.00%
WORKSITE_POSTAL_CODE: 17, 0.00%


**Result:** We can see that the most of the columns now have close to 0% missing values. Two columns however stand out - `H1B_DEPENDENT` & `WILLFUL_VIOLATOR`. Interestingly they have nearly identical number of missing values. Let's investigate this phenomenon further to see if we can observe any patterns / gain insights.

**B. Investigating missing values in `H1B_DEPENDENT` & `WILLFUL_VIOLATOR` columns:**

In [40]:
# Print out the first 15 indexes where the columns have nan values

hd_null_idx = np.where(h1b_df_clean10.H1B_DEPENDENT.isnull())[0]
print("First 15 null value index of H1B_DEPENDENT column:\n{}".format(hd_null_idx[:15]))
wv_null_idx= np.where(h1b_df_clean10.WILLFUL_VIOLATOR.isnull())[0]
print("First 15 null value index of WILLFUL_VIOLATOR column:\n{}".format(wv_null_idx[:15]))

First 15 null value index of H1B_DEPENDENT column:
[  8  41  73  96 197 293 417 480 578 588 589 613 631 703 717]
First 15 null value index of WILLFUL_VIOLATOR column:
[  8  41  73  96 197 293 417 480 578 588 589 613 631 703 717]


**Result:** This is very interesting! The first 20 indices at which the columns have missing (nan) values are identical. I wonder if there are more shared indices between the missing values of the two columns. To investigate that, we use the `set()` function to locate the intersecting indices between two lists.

In [41]:
shared_idx = list(set(hd_null_idx) & set(wv_null_idx))
len(shared_idx)
print("There are {} shared indices.".format(len(shared_idx)))

There are 14131 shared indices.


**Result:** WOW! There are 14131 shared indices that are shared between the missing values of the two columns. This is more than a coincidence. Let's index the cleaned dataframe with the above shared index to see if we can gain some insight. 

In [42]:
# # Accessing the cleaned dataframe (h1b_df_clean10) with the shared index from above
# # print out only the first n rows
h1b_df_clean10.iloc[shared_idx].head(3)

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,AGENT_REPRESENTING_EMPLOYER,JOB_TITLE,SOC_CODE,SOC_NAME,NAICS_CODE,TOTAL_WORKERS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMP,CHANGE_EMPLOYER,AMENDED_PETITION,FULL_TIME_POSITION,PREVAILING_WAGE,PW_UNIT_OF_PAY,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,WORKSITE_STATE,WORKSITE_POSTAL_CODE
32770,I-203-18211-042853,DENIED,2018-07-30,2018-08-01,E-3 Australian,2018-08-13,2018-10-10,"THE OFFICE PERFORMING ARTS + FILM, INC.",NY,10014,N,COMPANY MANAGER,27-1019,"ARTISTS AND RELATED WORKERS, ALL OTHER",71119.0,1,1,0,0,0,0,0,Y,37.02,Hour,1650.0,1650.0,Week,NaN,NaN,NY,10014
294915,I-203-18003-406124,CERTIFIED,2018-01-03,2018-01-09,E-3 Australian,2018-01-15,2020-01-15,"RECRUITERLY, INC.",CA,94607,Y,CHIEF FINANCIAL OFFICER,11-3031,FINANCIAL MANAGERS,561320.0,1,1,0,0,0,0,0,Y,89232.00,Year,89232.0,89232.0,Year,NaN,NaN,CA,94607
65539,I-203-17310-627935,CERTIFIED,2017-11-07,2017-11-14,E-3 Australian,2017-11-07,2019-11-06,"CNSDOSE, LLC",TX,70095,Y,CHIEF MEDICAL OFFICER,11-9111,MEDICAL AND HEALTH SERVICES MANAGERS,511210.0,1,1,0,0,0,0,0,Y,68370.00,Year,80000.0,80000.0,Year,NaN,NaN,TX,70095


**Result:** One thing stands out. The VISA class in this particular dataframe is either `E-3 Australian` or `H-1B1 Chile`. Let's investigate further by looking at value_counts() of the `VISA_CLASS` column of this dataframe.

In [43]:
# # Looking at value_counts() of VISA_CLASS column
nan_df = h1b_df_clean10.iloc[shared_idx]
nan_df_vc = nan_df.VISA_CLASS.value_counts()
nan_df_vc

E-3 Australian     12003
H-1B1 Singapore     1191
H-1B1 Chile          930
H-1B                   7
Name: VISA_CLASS, dtype: int64

**Result:** It looks like `H1B_DEPENDENT` & `WILLFUL_VIOLATOR` columns had missing values because the `VISA_CLASS` was different. However we should compare the above values with the total number of VISA CLASS observed from the entire dataset to be sure that the VISA CLASS is a factor in determining the values of `H1B_DEPENDENT` & `WILLFUL_VIOLATOR` columns. 

In [44]:
# # Using h1b_df_clean4_vc as the reference dataframe
h1b_df_clean10_vc = h1b_df_clean10.VISA_CLASS.value_counts()

us = nan_df_vc['H-1B']/h1b_df_clean10_vc['H-1B'] * 100
aus = nan_df_vc['E-3 Australian']/h1b_df_clean10_vc['E-3 Australian'] * 100
chile = nan_df_vc['H-1B1 Chile']/h1b_df_clean10_vc['H-1B1 Chile'] * 100
sing = nan_df_vc['H-1B1 Singapore']/h1b_df_clean10_vc['H-1B1 Singapore'] * 100

print("Percentage of VISA_CLASS counts compared to original dataset:\nH1-B: {:.2f}%\nAustralia: {:.2f}%\nChile: {:.2f}%\nSingapore: {:.2f}%".format(us, aus, chile, sing))


Percentage of VISA_CLASS counts compared to original dataset:
H1-B: 0.00%
Australia: 95.67%
Chile: 92.35%
Singapore: 94.08%


**Result:** So it looks like not all rows with **Australia, Chile, or Singapore** VISA CLASS had `H1B_DEPENDENT` & `WILLFUL_VIOLATOR` columns missing. Since we don't know whether this is a significant feature or not, let's impute the missing column values with ***N/A*** *(Not Applicable)* to retain as much information as possible. 

**C. Filling in missing values for `H1B_DEPENDENT` & `WILLFUL_VIOLATOR` columns:**

In [45]:
# # Replace the missing values with the string N/A
# # Must "add" the category first (see Note Below)

h1b_df_clean10['H1B_DEPENDENT'] = h1b_df_clean10['H1B_DEPENDENT'].cat.add_categories("N/A").fillna("N/A")
h1b_df_clean10['WILLFUL_VIOLATOR'] = h1b_df_clean10['WILLFUL_VIOLATOR'].cat.add_categories("N/A").fillna("N/A")

**NOTE** Because the columns are of `Category` dtype, must ["add"](https://stackoverflow.com/questions/32718639/pandas-filling-nans-in-categorical-data/36193135) the relevant category first before filling in the missing values. 

**E. Check Missing Values per Column after Value Imputation**

In [46]:
# # Commented Out to save Space
# display_missing_rows(h1b_df_clean10)
# # Dimensions of cleaned dataframe
h1b_df_clean10.shape

(654286, 32)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 2c'></a>
____

### 5c. Imputing missing values based on existing column relationship - STATE & POSTAL CODE

Most of the columns are now free of missing values and most of the missing values are now concentrated in `STATE` or `POSTAL_CODE` columns. Thankfully however, there is a *one* ***(state)***-to-*many* ***(postal_codes)*** relationship between states and postal codes. This relationship allows us to impute values for `STATE` column from the `POSTAL_CODE` column but not the other way around. As long as another row exists that contains the relationship between `STATE` column and `POSTAL_CODE` column, we will be able to successfully impute missing values for `STATE` column.




I will be doing the following steps to sucessfully impute `STATE` values based on `POSTAL_CODE` values. 

1. Make a dictionary from `STATE` and `POSTAL_CODE` data from the cleaned dataset (`h1b_df_clean10`)
   - Due to the **many-to-one relationship**, the keys of the dictionary must be 'many' (POSTAL_CODE) 
   - Make sure that the key:value pair of the dictionary is accurate (more on this <a href='#mode of values'>later</a>)
   
  
2. Using the dictionary, I will create a new column of STATES mapped from the dictionary.
3. Drop the original STATE column and rename the new STATE column accordingly. 
   
    
**(Summary):**
From *Postal_Code* data $\rightarrow$ fill in *State* data

**A. Investigating the missing values for `STATE` column:**

In [47]:
state_null = np.where(h1b_df_clean10.EMPLOYER_STATE.isnull())[0]
state_null_df = h1b_df_clean10.loc[state_null]
state_null_df.shape

print("There are {} rows with missing STATE values".format(state_null_df.shape[0]))


There are 67 rows with missing STATE values


<a id='mode of values'></a>
**B. Make a dictionary from `STATE` and `POSTAL CODE` columns**


Before blindly creating a dictionary from these two columns, we need to: 
1. sort the keys 
2. verify that the key:value pair is valid 
    - **Note**: Due to human error, zipcode and state could have been inputted incorrectly at the time of dataset creation. If for some reason after sorting by **keys**, the incorrect **value** is lexically behind the correct **value**, then the dictionary will have an incorrect key:value pair and the error will propagate. To prevent these edge cases from affecting our data integrity, we rely on the wisdom of the masses. Basically, we treat the *mode* of the **value** as a valid **value**. 

###### Sort the Keys

In [48]:
# # Create a 'state df' that only contains two columns of our interest
# # Drop any nan values to prevent nans getting into the dictionary
state_df = h1b_df_clean10[['EMPLOYER_STATE', 'EMPLOYER_POSTAL_CODE']].dropna(how='any')

# # sort by postal code (our keys)
state_df = state_df.sort_values(by='EMPLOYER_POSTAL_CODE')

# # # of Unique Postal Codes 
u_num = state_df.EMPLOYER_POSTAL_CODE.unique().shape[0]
print("Number of Unique Postal Codes: {}".format(u_num))
# # Example Case of how wrong key:value pair can be introduced into the dictionary
state_df.loc[state_df.EMPLOYER_POSTAL_CODE == '98052'].sort_values(by='EMPLOYER_STATE',ascending=False).head(3)

# # Here, we can see that the correct State corresponding to a zipcode of 98052 is 'WA'. 
# # However, if we sort incorrectly, the corresponding value pair for the key 98052 will be incorrectly set to 'WV'

Number of Unique Postal Codes: 10771


,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE
583613,WV,98052
638401,WA,98052
213485,WA,98052


###### Function that will be applied to the Series

In [49]:
def apply_mode(inp):
    inp_df = state_df[state_df.EMPLOYER_POSTAL_CODE == inp]          # Filters dataframe by the Postal Code
    output = inp_df.EMPLOYER_STATE.value_counts().index.tolist()[0]  # Value counts returns the sorted index
    
    return output 


## QUESTION: ASK BEN ABOUT PERFORMANCE (of .apply())

- apply method is very slow: any way to increase the performance??

In [50]:
# # Make a new dataframe and apply the function to the column (EMPLOYER_STATE)
state_df2 = state_df.copy()

# # Map STATE column based on POSTAL CODE values 
state_df2.EMPLOYER_STATE = state_df2.EMPLOYER_POSTAL_CODE.apply(apply_mode)  
# # outputs mode to employer state column

# # PERFORMANCE IS VERY SLOW!! 

###### Verify the validity of key:value pair

In [51]:
# # Verify that the apply function worked
# # This particular example (state_df2) should only have 'WA' as output

# state_df.loc[state_df.EMPLOYER_POSTAL_CODE == '98052'].EMPLOYER_STATE.value_counts()

state_df2.loc[state_df2.EMPLOYER_POSTAL_CODE == '98052'].EMPLOYER_STATE.value_counts()


WA    6453
Name: EMPLOYER_STATE, dtype: int64

###### Make the dictionary

In [52]:

# # Making a dictionary with state_df2
# # Make sure that POSTAL_CODE is keys()!

state_dict = dict(zip(state_df2.EMPLOYER_POSTAL_CODE, state_df2.EMPLOYER_STATE))

**C. Create a New Dataframe with the new column mapped from the dictionary + Rearrange Columns**

In [53]:
# # Create a copy and make a new column + Rearrange Columns
h1b_df_clean11 = h1b_df_clean10.copy()
h1b_df_clean11['EMPLOYER_STATE_MAPPED'] = h1b_df_clean11['EMPLOYER_POSTAL_CODE'].map(state_dict)

h1b_df_clean11 = h1b_df_clean11[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS', 'EMPLOYMENT_START_DATE', 'EMPLOYMENT_END_DATE',
       'EMPLOYER_NAME', 'EMPLOYER_STATE_MAPPED', 'EMPLOYER_STATE','EMPLOYER_POSTAL_CODE', 'AGENT_REPRESENTING_EMPLOYER',
       'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMP', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'WORKSITE_STATE',
       'WORKSITE_POSTAL_CODE']]

**D. Check the Validity of Imputations**

In [54]:
mapped_series = h1b_df_clean11[['EMPLOYER_STATE_MAPPED']].EMPLOYER_STATE_MAPPED 
orig_series = h1b_df_clean11[['EMPLOYER_STATE']].EMPLOYER_STATE

In [55]:
# # Find the number of / index at where the Mapped and the Original Values differ

num_modified = len(np.where(mapped_series != orig_series)[0]) 
print("{} values have been modified after mapping".format(num_modified))

mod_idx = np.where(mapped_series != orig_series)[0]
h1b_df_clean11.loc[mod_idx,['EMPLOYER_STATE_MAPPED','EMPLOYER_STATE','EMPLOYER_POSTAL_CODE']].head(5)
                   

463 values have been modified after mapping


,EMPLOYER_STATE_MAPPED,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE
587,NE,MO,68114
731,NE,MO,68114
2385,TX,MA,75024
3281,TX,OH,77042
3936,NY,NJ,11106


**RESULT**: 463 values differed between the mapped and the original columns. This means that potentially 463 values have been inputted incorrectly. We can easily check the validity of the imputation with a quick google search. For example at index 587, the state for Postal Code '68114' is in fact Omaha, NE (Nebraska), not MO (Missouri). Using the mode of the frequency, we were able to catch incorrect inputs and modify them accordingly, thereby increasing our data integrity! 

**E. Drop the original `EMPLOYER STATE` column and check dimensions**

In [56]:
# # Drop Original and Rearrange Columns
h1b_df_clean12 = h1b_df_clean11.drop('EMPLOYER_STATE', axis=1)
h1b_df_clean12.shape

(654286, 32)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 2d'></a>
____

### 5d. POSTAL CODES that are *"weird"*

Let's check our missing values again using the `display_missing_rows` function.




In [57]:
display_missing_rows(h1b_df_clean12)

MISSING ROWS per COLUMN
CASE_NUMBER: 0, 0.00%
CASE_STATUS: 0, 0.00%
CASE_SUBMITTED: 1, 0.00%
DECISION_DATE: 0, 0.00%
VISA_CLASS: 0, 0.00%
EMPLOYMENT_START_DATE: 3, 0.00%
EMPLOYMENT_END_DATE: 6, 0.00%
EMPLOYER_NAME: 13, 0.00%
EMPLOYER_STATE_MAPPED: 71, 0.01%
EMPLOYER_POSTAL_CODE: 12, 0.00%
AGENT_REPRESENTING_EMPLOYER: 10, 0.00%
JOB_TITLE: 0, 0.00%
SOC_CODE: 3, 0.00%
SOC_NAME: 3, 0.00%
NAICS_CODE: 1, 0.00%
TOTAL_WORKERS: 0, 0.00%
NEW_EMPLOYMENT: 0, 0.00%
CONTINUED_EMPLOYMENT: 0, 0.00%
CHANGE_PREVIOUS_EMPLOYMENT: 0, 0.00%
NEW_CONCURRENT_EMP: 0, 0.00%
CHANGE_EMPLOYER: 0, 0.00%
AMENDED_PETITION: 0, 0.00%
FULL_TIME_POSITION: 2, 0.00%
PREVAILING_WAGE: 0, 0.00%
PW_UNIT_OF_PAY: 1, 0.00%
WAGE_RATE_OF_PAY_FROM: 0, 0.00%
WAGE_RATE_OF_PAY_TO: 0, 0.00%
WAGE_UNIT_OF_PAY: 3, 0.00%
H1B_DEPENDENT: 0, 0.00%
WILLFUL_VIOLATOR: 0, 0.00%
WORKSITE_STATE: 4, 0.00%
WORKSITE_POSTAL_CODE: 17, 0.00%


**RESULT:** Interstingly, the missing values in newly mapped `EMPLOYER_STATE_MAPPED` column increased to 71 values from 67! This is probably due to the fact that `EMPLOYER_POSTAL_CODE` has 12 missing values as well. Let's drop the rows at which `EMPLOYER_POSTAL_CODE` values are null and re-investigate the dataframe.

**A. Drop missing values from `EMPLOYER_POSTAL_CODE` column**

In [58]:
# # Drop the Rows where EMPLOYER_POSTAL_CODE is null

# # Getting the index where the column is null 
zipcode_null_idx = np.where(h1b_df_clean12.EMPLOYER_POSTAL_CODE.isnull())[0]

# # Drop the null indices and reset the index! 
h1b_df_clean13 = h1b_df_clean12.drop(zipcode_null_idx)
h1b_df_clean14 = h1b_df_clean13.reset_index(drop=True)

h1b_df_clean14.shape


(654274, 32)

In [59]:
display_missing_rows(h1b_df_clean14)

MISSING ROWS per COLUMN
CASE_NUMBER: 0, 0.00%
CASE_STATUS: 0, 0.00%
CASE_SUBMITTED: 1, 0.00%
DECISION_DATE: 0, 0.00%
VISA_CLASS: 0, 0.00%
EMPLOYMENT_START_DATE: 3, 0.00%
EMPLOYMENT_END_DATE: 6, 0.00%
EMPLOYER_NAME: 12, 0.00%
EMPLOYER_STATE_MAPPED: 59, 0.01%
EMPLOYER_POSTAL_CODE: 0, 0.00%
AGENT_REPRESENTING_EMPLOYER: 10, 0.00%
JOB_TITLE: 0, 0.00%
SOC_CODE: 3, 0.00%
SOC_NAME: 3, 0.00%
NAICS_CODE: 0, 0.00%
TOTAL_WORKERS: 0, 0.00%
NEW_EMPLOYMENT: 0, 0.00%
CONTINUED_EMPLOYMENT: 0, 0.00%
CHANGE_PREVIOUS_EMPLOYMENT: 0, 0.00%
NEW_CONCURRENT_EMP: 0, 0.00%
CHANGE_EMPLOYER: 0, 0.00%
AMENDED_PETITION: 0, 0.00%
FULL_TIME_POSITION: 2, 0.00%
PREVAILING_WAGE: 0, 0.00%
PW_UNIT_OF_PAY: 1, 0.00%
WAGE_RATE_OF_PAY_FROM: 0, 0.00%
WAGE_RATE_OF_PAY_TO: 0, 0.00%
WAGE_UNIT_OF_PAY: 3, 0.00%
H1B_DEPENDENT: 0, 0.00%
WILLFUL_VIOLATOR: 0, 0.00%
WORKSITE_STATE: 4, 0.00%
WORKSITE_POSTAL_CODE: 17, 0.00%


**RESULT:** After dropping the missing value rows from `EMPLOYER_POSTAL_CODE`, there are still  59 missing values left in the `EMPLOYER_STATE_MAPPED` column. This seems a bit high, so let's index into the dataframe to see if anything pops out. 

**B. Investigating the missing values in `EMPLOYER_STATE_MAPPED` column**

In [60]:
mapped_state_null_idx = np.where(h1b_df_clean14.EMPLOYER_STATE_MAPPED.isnull())[0]
mapped_df = h1b_df_clean14.loc[mapped_state_null_idx,['EMPLOYER_NAME', 'EMPLOYER_STATE_MAPPED','EMPLOYER_POSTAL_CODE']]
mapped_df.head(15)
# mapped_df.EMPLOYER_NAME.value_counts()


,EMPLOYER_NAME,EMPLOYER_STATE_MAPPED,EMPLOYER_POSTAL_CODE
9290,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
13167,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
16092,COMPUTERTALK TECHNOLOGY INC,NaN,L3T7M8
17175,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
41583,GOPC PTY LTD,NaN,6850
44535,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
48487,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
52499,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
56419,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008
60235,GOLDEN OPPORTUNITY VENTURES INC.,NaN,6008


**RESULT:** I only printed out the first 15 rows but we can begin to see a pattern here. First of all, some of the zipcodes are in strings, which are indicative of Canadian addresses. Second of all, there are only a handful of companies that have missing `STATE` values. (Quick view of `mapped_df.EMPLOYER_NAME.value_counts()` shows this.) A quick google search also reveals that **Lululemon USA Inc.** has its headquarters in Vancouver, Canada. This signifies that these rows with missing `STATE` columns are probably companies outside of United States. Thus, we will impute these values accordingly by replacing nan values with **"Other"**.

**C. Replacing nan values with "Other"**

In [61]:
h1b_df_clean15 = h1b_df_clean14.copy()
h1b_df_clean15.EMPLOYER_STATE_MAPPED = h1b_df_clean15.EMPLOYER_STATE_MAPPED.replace(np.nan,'Other')

# # Checking to see that replacement occurred (dataframe)
t_df = h1b_df_clean15.loc[mapped_state_null_idx,['EMPLOYER_NAME', 'EMPLOYER_STATE_MAPPED','EMPLOYER_POSTAL_CODE']]
# t_df

**D. Check replacement worked by calling `display_missing_rows()`**

In [62]:
# # Check missing values again 
# # Commented out for space constraints
# display_missing_rows(h1b_df_clean15)

<a href='#Table of Contents'>Back to Table of Contents</a>

<a id='Dataset Cleaning 2e'></a>
____

### 5e. Final Missing Value Handling

As we saw above, there are now only a handful of missing values left. Since there are only a few rows with missing values left compared to the original dataset, it would just make sense to drop any rows that now contain any missing values. We will perform this operation and compare the final dataframe dimensions with that of the original dataframe. 

[How to Drop nans - Comment from WMcKinney himself!](https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan/13434501#13434501)




**A. Dropping nan values from dataframe**

In [63]:
# # Drop missing values
h1b_df_clean16 = h1b_df_clean15.dropna()
h1b_df_clean_final = h1b_df_clean16.reset_index(drop=True)

**B. Verification and Comparision with original dataframe**

In [64]:
# # Commented out due to space constraints
# display_missing_rows(h1b_df_clean_final)

# # Compare with Original Dataframe
o_shape = h1b_df.shape
f_shape = h1b_df_clean_final.shape

print("Compared with the original dataframe of dimensions {}, we are now left with a dimension of {}.".format(o_shape, f_shape))
print("Compared with the original dataframe, we are left with {:0.4f}% of rows.".format(f_shape[0]/o_shape[0] * 100))



Compared with the original dataframe of dimensions (654360, 50), we are now left with a dimension of (654216, 32).
Compared with the original dataframe, we are left with 99.9780% of rows.


#### END OF NOTEBOOK
<a href='#Top'>Back to Top</a>